# CMS GIWAXS plotting notebook - plotting single images from loaded zarr datasets
# A1-3 Sample Set

## Imports

In [1]:
# Imports:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import xarray as xr
from tqdm.auto import tqdm

# Choose a colormap:
cmap = plt.cm.turbo
cmap.set_bad('black')

## Define & check paths

In [2]:
def select_attrs(data_arrays_iterable, selected_attrs_dict):
    """
    Selects data arrays whose attributes match the specified values.

    Parameters:
    data_arrays_iterable: Iterable of xarray.DataArray objects.
    selected_attrs_dict: Dictionary where keys are attribute names and 
                         values are the attributes' desired values.

    Returns:
    List of xarray.DataArray objects that match the specified attributes.
    """    
    sublist = list(data_arrays_iterable)
    
    for attr_name, attr_values in selected_attrs_dict.items():
        sublist = [da.copy() for da in sublist if da.attrs[attr_name] in attr_values]
                
    return sublist

def fold_image(data_array, fold_axis, progress_bar=False):
    """
    Method to fold image along a specified axis.
    
    Parameters:
    - data_array (xarray DataArray): The DataArray to fold
    - fold_axis (str): The axis along which to fold the image
    
    Returns:
    - xarray DataArray: The folded image
    """
    # Filter data for fold_axis >= 0 and fold_axis <= 0
    positive_data = data_array.where(data_array[fold_axis] >= 0, drop=True)
    negative_data = data_array.where(data_array[fold_axis] <= 0, drop=True)
    
    # Reverse negative_data for easier comparison
    negative_data = negative_data.reindex({fold_axis: negative_data[fold_axis][::-1]})
    
    # Find the maximum coordinate of the shorter quadrant (positive_data)
    max_positive_coord = float(positive_data[fold_axis].max())
    
    # Find the equivalent coordinate in the negative_data
    abs_diff = np.abs(negative_data[fold_axis].values + max_positive_coord)
    
    # Minimize the difference
    min_diff_idx = np.argmin(abs_diff)
    
    # Check if the lengths are equivalent
    len_pos = len(positive_data[fold_axis])
    len_neg = len(negative_data[fold_axis][:min_diff_idx+1])
    
    if len_pos != len_neg:
        # Adjust the coordinate range for negative_data
        for i in range(1, 4):  # Check 3 neighbors
            new_idx = min_diff_idx + i
            len_neg = len(negative_data[fold_axis][:new_idx+1])
            if len_pos == len_neg:
                min_diff_idx = new_idx
                break
                
    # Crop the negative_data to match positive_data length
    negative_data_cropped = negative_data.isel({fold_axis: slice(0, min_diff_idx+1)})
    
    # Prepare the new data array
    new_data = xr.zeros_like(positive_data)
    
    # Fold the image
    if progress_bar:
        for i in tqdm(range(len(positive_data[fold_axis]))):
            pos_val = positive_data.isel({fold_axis: i}).values
            neg_val = negative_data_cropped.isel({fold_axis: i}).values

            # Pixel comparison and choosing 
            new_data[i] = np.where(((pos_val == 0) | (np.isnan(pos_val))) & (neg_val > 1-1e-6),
                                   neg_val, 
                                   pos_val)
    else:
        for i in range(len(positive_data[fold_axis])):
            pos_val = positive_data.isel({fold_axis: i}).values
            neg_val = negative_data_cropped.isel({fold_axis: i}).values

            # Pixel comparison and choosing 
            new_data[i] = np.where(((pos_val == 0) | (np.isnan(pos_val))) & (neg_val > 1-1e-6),
                                   neg_val, 
                                   pos_val) 
            
        # Pixel comparison and averaging
        # new_data[i] = np.where(
        #     (pos_val > 0) & (neg_val > 0), 
        #     (pos_val + neg_val) / 2,
        #     np.where(((pos_val == 0) | (np.isnan(pos_val))) & (neg_val > 0),
        #              neg_val, 
        #              pos_val)
        # )


        
    # Append residual data from the longer quadrant if exists
    if len(negative_data[fold_axis]) > min_diff_idx+1:
        residual_data = negative_data.isel({fold_axis: slice(min_diff_idx+1, None)})
        residual_data[fold_axis] = np.abs(residual_data[fold_axis])
        new_data = xr.concat([new_data, residual_data], dim=fold_axis)
        
    # Update data_array with the folded image
    data_array = new_data.sortby(fold_axis)
    
    # Inherit coordinates and metadata attributes from the original data_array
    data_array.attrs = data_array.attrs.copy()
    data_array.attrs['fold_axis'] = fold_axis  # Add 'fold_axis' attribute

    # Ensure all original coordinates are retained in the new data_array
    for coord in data_array.coords:
        if coord not in data_array.coords:
            data_array = data_array.assign_coords({coord: data_array[coord]})

    return data_array

In [3]:
# I like pathlib for its readability & checkability, it's also necessary for the loadSeries function later on
# Replace the paths with the ones relevant to your data, you can use the ".exists()" method to make sure you defined a path correctly
propPath = pathlib.Path('/nsls2/data/cms/proposals/2023-2/pass-311415')  # The proposals path is a good place to store large data (>1 TB space?)
outPath = propPath.joinpath('AL_processed_data')

samplesPath = outPath.joinpath('ex_situ_zarrs_v2/bcy651')

In [4]:
# List the files inside a folder
sorted([f.name for f in samplesPath.iterdir()])  # a way to list just the filenames and not the whole path

['A1-3set_caked_stitched.zarr',
 'A1-3set_raw_stitched.zarr',
 'A1-3set_recip_stitched.zarr']

## Cartesian plots

### Load dataset

In [ ]:
filename = 'A1-3set_raw_stitched.zarr'
DS = xr.open_zarr(samplesPath.joinpath(filename))
DS = DS.where(DS>1e-6)
DS

### 2D reciprocal space images

In [ ]:
# 2D raw pixels cartesian plots
x_min = 150
x_max = 850
y_min = 100
y_max = 650

# savePath = outPath.joinpath('A1-3set_waxs/bcy651/recip_plots_v1')

selected_attrs_dict = {'film':['D18p1']}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)

for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(pix_x=slice(x_min, x_max), pix_y=slice(y_min, y_max))
    
    real_min = float(sliced_DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(sliced_DA.compute().quantile(0.995))   
    
    # Plot
    ax = sliced_DA.plot.imshow(origin='upper', cmap=cmap, norm=plt.Normalize(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    ax.axes.set(aspect='equal', title=f'Stitched detector image: {DA.film}, $\\alpha_i$ = {float(DA.incident_angle[2:])}°',
                xlabel='x pixels [num]', ylabel='y pixels [num]')
    ax.figure.set(tight_layout=True, dpi=130)
    
    # ax.figure.savefig(savePath.joinpath(f'{DA.film}_qxy{qxy_min}to{qxy_max}_qz{qz_min}to{qz_max}_{DA.incident_angle}.png'), dpi=150)
    
    plt.show()
    plt.close('all')

In [ ]:
# 2D reciprocal space cartesian plots
qxy_min = -1.1
qxy_max = 2.1
qz_min = -0.001
qz_max = 2.2

savePath = outPath.joinpath('A1-3set_waxs/bcy651/recip_plots_v1')

selected_attrs_dict = {}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)

for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))
    
    real_min = float(sliced_DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(sliced_DA.compute().quantile(0.995))   
    
    # Plot
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), interpolation='antialiased', figsize=(5.5,3.3))
    ax.colorbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
    ax.axes.set(aspect='equal', title=f'Cartesian Plot: {DA.film}, {float(DA.incident_angle[2:])}° Incidence',
                xlabel='q$_{xy}$ [Å$^{-1}$]', ylabel='q$_z$ [Å$^{-1}$]')
    ax.figure.set(tight_layout=True, dpi=130)
    
    ax.figure.savefig(savePath.joinpath(f'{DA.film}_qxy{qxy_min}to{qxy_max}_qz{qz_min}to{qz_max}_{DA.incident_angle}.png'), dpi=150)
    
    # plt.show()
    plt.close('all')

In [ ]:
# # A way to save data as csv files
# for DA in tqdm(DS.data_vars.values()):
#     # qxy columns, qz rows
#     DA.to_pandas().to_csv(outPath.joinpath('PM6-Y6_waxs', f'cartesian-2D_{DA.polymer}-{DA.weight_percent}_{DA.incident_angle}_{DA.scan_id}.csv'))

### Yoneda Check

In [ ]:
def qz(wavelength, alpha_crit, alpha_incidents):
    qz_inv_meters = ((4 * np.pi) / (wavelength)) * (np.sin(np.deg2rad((alpha_incidents + alpha_crit)/2)))
    # qz_inv_meters = ((4 * np.pi) / (wavelength)) * (np.sin(np.deg2rad(alpha_crit)) + np.sin(np.deg2rad(alpha_incidents)))
    qz_inv_angstroms = qz_inv_meters / 1e10
    return qz_inv_angstroms


wavelength = 9.762535309700809e-11  # 12.7 keV

alpha_crit = 0.11  # organic film critical angle
alpha_incidents = np.array([0.1, 0.12, 0.14])

yoneda_angles = alpha_incidents + alpha_crit

qz(wavelength, alpha_crit, alpha_incidents)

In [ ]:
# Yoneda peak linecut check
qxy_min = 0.22
qxy_max = 2
qz_min = -0.02
qz_max = 0.06

selected_attrs_dict = {'film': ['D18p1']}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)
for DA in tqdm(selected_DAs):
    # Slice data for selected q ranges (will need to rename q_xy if dimensions are differently named)
    sliced_DA = DA.sel(q_xy=slice(qxy_min, qxy_max), q_z=slice(qz_min, qz_max))
    qz_integrated_DA = sliced_DA.sum('q_xy')
    
    # Plot
    qz_integrated_DA.plot.line(label=DA.incident_angle)
    
plt.legend()
plt.grid(visible=True, which='major', axis='x')
plt.show()
plt.close('all')

### 1D Qxy or Qz integrated lincuts

## Polar plots

### Load dataset

In [5]:
filename = 'A1-3set_caked_stitched.zarr'
DS = xr.open_zarr(samplesPath.joinpath(filename))
DS = DS.where(DS>1e-6)
DS

<xarray.Dataset>
Dimensions:  (chi: 180, qr: 1000)
Coordinates:
  * chi      (chi) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * qr       (qr) float64 0.0009925 0.004228 0.007464 ... 3.227 3.23 3.234
Data variables: (12/30)
    1116580  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116581  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116582  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116586  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116587  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116588  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    ...       ...
    1116628  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116629  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116630  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116634  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116635  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>
    1116636  (chi, qr) float64 dask.array<chunksize=(90, 500), meta=np.ndarray>

In [6]:
### Apply a sin chi correction
sin_chi_DA = np.sin(np.radians(np.abs(DS.chi)))

corr_DS = DS.copy()
# corr_DS = corr_DS * sin_chi_DA  # This works mathematically, but does not preserve attributes
for var in corr_DS.data_vars:
    corrected = corr_DS[var] * sin_chi_DA
    corr_DS[var].values = corrected.values
    
corr_DS

<xarray.Dataset>
Dimensions:  (chi: 180, qr: 1000)
Coordinates:
  * chi      (chi) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * qr       (qr) float64 0.0009925 0.004228 0.007464 ... 3.227 3.23 3.234
Data variables: (12/30)
    1116580  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116581  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116582  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116586  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116587  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116588  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    ...       ...
    1116628  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116629  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116630  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116634  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116635  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    1116636  (chi, qr) float64 nan nan nan nan nan nan ... nan nan nan nan nan

In [7]:
### Fold sin(chi) corrected dataset

folded_corr_DAs = []
for DA in tqdm(corr_DS.data_vars.values()):
    folded_corr_DAs.append(fold_image(DA, 'chi'))

  0%|          | 0/30 [00:00<?, ?it/s]

### 2D caked images

#### Unfolded raw(chi) intensity

In [ ]:
# Polar plots, for raw(chi) intensities

# Set chi range: Full range
chi_min = -90
chi_max = 90
q_min = None
q_max = None

savePath = outPath.joinpath('A1-3set_waxs/bcy651/caked_plots_v1')
    
selected_attrs_dict = {}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)

for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min, q_max))
    
    # Set color limits
    real_min = float(sliced_DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(sliced_DA.compute().quantile(0.995))       
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(5,4), interpolation='antialiased')  # plot, optional parameter interpolation='antialiased' for image smoothing
    ax.axes.set(title=f'Polar Plot: {DA.film}, {float(DA.incident_angle[2:])}° Incidence, Raw Intensity')
    ax.colorbar.set_label('Raw Intensity [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    # plt.show()  # Comment to mute plotting output
    
    # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    ax.figure.savefig(savePath.joinpath(f'rawchi_{DA.film}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')

#### Folded raw(chi) intensity

In [ ]:
# Polar plots, for FOLDED raw(chi) intensities

# Set chi range: Full range
chi_min = None
chi_max = None
q_min = None
q_max = None

savePath = outPath.joinpath('A1-3set_waxs/caked_plots_v1')

selected_attrs_dict = {'film':['A3']}
selected_DAs = select_attrs(DS.data_vars.values(), selected_attrs_dict)
    
for DA in tqdm(selected_DAs):
    DA = fold_image(DA, 'chi')

    # Set color limits
    real_min = float(DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(DA.compute().quantile(0.995))  
    
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min, q_max))     
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(5,4), interpolation='antialiased')  # plot, optional parameter interpolation='antialiased' for image smoothing
    ax.axes.set(title=f'Folded Polar Plot: {DA.film}, {float(DA.incident_angle[2:])}° Incidence, Raw Intensity')
    ax.colorbar.set_label('Raw Intensity [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    plt.show()  # Comment to mute plotting output
    
    # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    # ax.figure.savefig(savePath.joinpath(f'rawchi-folded_{DA.film}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')

#### Unfolded sin(chi) intensity

In [ ]:
# Polar plots, for sin(chi) intensities

# Set chi range: Full range
chi_min = -90
chi_max = 90
q_min = None
q_max = None

savePath = outPath.joinpath('A1-3set_waxs/caked_plots_v1')
    
for DA in tqdm(selected_corr_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min, q_max))
    
    # Set color limits
    real_min = float(sliced_DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(sliced_DA.compute().quantile(0.995))       
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(5,4), interpolation='antialiased')  # plot, optional parameter interpolation='antialiased' for image smoothing
    ax.axes.set(title=f'Polar Plot: {DA.film}, {float(DA.incident_angle[2:])}° Incidence, sin($\chi$) Corrected')
    ax.colorbar.set_label('Intensity * sin($\chi$) [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    plt.show()  # Comment to mute plotting output
    
    # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    # ax.figure.savefig(savePath.joinpath(f'sinchi_{DA.film}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')

#### Folded sin(chi) intensity

In [ ]:
# Polar plots, for FOLDED sin(chi) intensities

# Set chi range: Full range
chi_min = 10
chi_max = 82
q_min = 0.2
q_max = 2.04

# Set savePath
savePath = outPath.joinpath('A1-3set_waxs/bcy651/caked_plots_v1')

selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)
    
for DA in tqdm(selected_DAs):
    # Slice dataarray to select plotting region 
    sliced_DA = DA.sel(chi=slice(chi_min,chi_max), qr=slice(q_min, q_max))
    
    # Set color limits
    real_min = float(sliced_DA.compute().quantile(0.01))
    cmin = 1 if real_min < 1 else real_min

    cmax = float(sliced_DA.compute().quantile(0.995))       
    
    # Plot sliced dataarray
    ax = sliced_DA.plot.imshow(origin='upper', cmap=cmap, norm=plt.Normalize(cmin, cmax), figsize=(5,4))  # plot, optional parameter interpolation='antialiased' for image smoothing
    ax.axes.set(title=f'Folded polar plot: {DA.film}, $\\alpha_i$ = {float(DA.incident_angle[2:])}°, sin($\chi$) corrected')
    ax.colorbar.set_label('Intensity * sin($\chi$) [arb. units]', rotation=270, labelpad=15)  # set colorbar label & parameters 
    ax.axes.set(xlabel='q$_r$ [Å$^{-1}$]', ylabel='$\chi$ [°]')  # set title, axis labels, misc
    ax.figure.set(tight_layout=True, dpi=130)  # Adjust figure dpi & plotting style
    
    plt.show()  # Comment to mute plotting output
    
    # Uncomment below line and set savepath/savename for saving plots, I usually like to check 
    # ax.figure.savefig(savePath.joinpath(f'sinchi-folded_{DA.film}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    plt.close('all')

### 1D linecuts along chi

In [8]:
import matplotlib.ticker as ticker

In [9]:
import matplotlib.colors as mcolors

In [10]:
def darken_colors(color_array, factor):
    """
    Darken an array of RGB colors where 0 is white and 1 is black.
    
    :param color_array: An array of RGB colors in the range [0, 1] where 0 is white and 1 is black
    :param factor: A float between 0 and 1, where 0 has no change and 1 is fully black
    :return: The darkened color array
    """
    darkened_colors = color_array + (1 - color_array) * factor
    return np.clip(darkened_colors, 0, 1)  # Ensures values stay within valid range

In [11]:
# colors = plt.cm.tab20c_r(np.linspace(0,1,20))[:16][::2]  # for 2 incident angles
colors = plt.cm.tab20c_r(np.linspace(0,1,20))[:16]  # for 4 incident angles
colors = darken_colors(colors, -0.5)

In [12]:
# Indices of rows to be deleted
indices_to_delete = [0, 4, 8, 12]

# Deleting the specified rows
arr_modified = np.delete(colors, indices_to_delete, axis=0)

In [13]:
# colors = arr_modified
# # Create a custom colormap
# custom_cmap = mcolors.ListedColormap(colors)

# # Create a scalar mappable object with the normalization and colormap
# norm = mcolors.Normalize(vmin=0, vmax=colors.shape[0])
# scalar_mappable = plt.cm.ScalarMappable(norm=norm, cmap=custom_cmap)

# # Plot the colorbar
# plt.colorbar(scalar_mappable, ticks=range(colors.shape[0]), spacing='proportional')
# plt.title('Custom Colorbar')
# plt.show()

In [40]:
# Set lims
oop_chi_min = 10
oop_chi_max = 30

ip_chi_min = 65
ip_chi_max = 82

q_min = 0.1
q_max = 2.04
# q_min = 0.2
# q_max = 0.8

# Set savePath
savePath = outPath.joinpath('A1-3set_waxs/bcy651/stacked_linecut_plots_v1')

# selected_attrs_dict = {'incident_angle':['th0.080', 'th0.140'],
#                        'material':['Y6']}
selected_attrs_dict = {'film':['A1', 'A2', 'A3']}
# selected_attrs_dict = {}

# group_size = 2
# group_size = 4
group_size = 3

selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)
DA_sublists = [selected_DAs[i:i + group_size] for i in range(0, len(selected_DAs), group_size)]

# colors = plt.cm.tab20c_r(np.linspace(0,1,20))[:16][::2]  # for 2 incident angles
colors = plt.cm.tab20c_r(np.linspace(0,1,20))[:16]  # for 4 incident angles
colors = darken_colors(colors, -0.5)

indices_to_delete = [0, 4, 8, 12]  # for 3 incident angles
colors = np.delete(colors, indices_to_delete, axis=0)

for DA_sublist in tqdm(DA_sublists):
    # Sort by incident angle:
    DA_sublist = sorted(DA_sublist, key = lambda x: x.incident_angle)
    
    # Define the figure and subplots
    fig, axs = plt.subplots(4, 1, figsize=(6, 8), sharex=True, dpi=120)

    # Adjust the spacing between plots
    fig.subplots_adjust(hspace=0.005)  # Adjust this value as needed to minimize white space
    
    for i, DA in enumerate(DA_sublist):        
        # Plotting each line in its subplot
        # OOP
        OOP_linecut_DA = DA.sel(chi=slice(oop_chi_min, oop_chi_max), qr=slice(q_min,q_max)).integrate('chi') / np.abs(oop_chi_max - oop_chi_min)
        OOP_linecut_DA.plot.line(color=colors[i], ax=axs[0], label=f'$\\alpha_i$ = {float(DA.incident_angle[2:])}°')

#         # Save OOP linecut DA as text file:
#         txtPath = outPath.joinpath('A1-3set_waxs/bcy651/text_files/OOP_linecuts')
#         npy_data = np.vstack((OOP_linecut_DA.qr.data, OOP_linecut_DA.data)).T
#         np.savetxt(txtPath.joinpath(f'{DA.film}_{DA.incident_angle}.txt'), npy_data)
        
        # Diagonal
        Diagonal_linecut_DA = DA.sel(chi=slice(oop_chi_max, ip_chi_min), qr=slice(q_min,q_max)).integrate('chi') / np.abs(ip_chi_min - oop_chi_max)
        Diagonal_linecut_DA.plot.line(color=colors[i+(group_size*1)], ax=axs[1], label=f'$\\alpha_i$ = {float(DA.incident_angle[2:])}°')

        # IP
        IP_linecut_DA = DA.sel(chi=slice(ip_chi_min, ip_chi_max), qr=slice(q_min,q_max)).integrate('chi') / np.abs(ip_chi_max - ip_chi_min)
        IP_linecut_DA.plot.line(color=colors[i+(group_size*2)], ax=axs[2], label=f'$\\alpha_i$ = {float(DA.incident_angle[2:])}°')

        # Full
        Full_linecut_DA = DA.sel(chi=slice(oop_chi_min, ip_chi_max), qr=slice(q_min,q_max)).integrate('chi') / 90
        Full_linecut_DA.plot.line(color=colors[i+(group_size*3)], ax=axs[3], label=f'$\\alpha_i$ = {float(DA.incident_angle[2:])}°')

    # Add legends and gridlines to each subplot
    for ax in axs:
        ax.grid(True, which='major', axis='x')  # Add gridlines
        ax.xaxis.set_major_locator(ticker.AutoLocator())  # Set major tick locator
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())  # Set minor tick locator
        ax.set_ylabel('')

    axs[0].text(0.5, 0.88, f'Out-of-plane, {oop_chi_min}-{oop_chi_max}°' + r' $\chi$', fontsize=12,
                horizontalalignment='center', transform=axs[0].transAxes)
    axs[1].text(0.5, 0.88, f'Diagonal, {oop_chi_max}-{ip_chi_min}°' + r' $\chi$', fontsize=12,
                horizontalalignment='center', transform=axs[1].transAxes)
    axs[2].text(0.5, 0.88, f'In-plane, {ip_chi_min}-{ip_chi_max}°' + r' $\chi$', fontsize=12,
                horizontalalignment='center', transform=axs[2].transAxes)
    axs[3].legend(loc='upper center', title=f'Full, {oop_chi_min}-{ip_chi_max}°' + r' $\chi$',
                  fontsize=10, title_fontsize='large')
       

    # Add axis labels
    axs[-1].set_xlabel('$q_r$ $[Å^{-1}]$', fontsize=12)
    fig.text(0.01, 0.5, 'Integrated intensities [arb. units]', va='center', rotation='vertical', fontsize=12)

    # fig.suptitle(f'{DA.material} {DA.solvent}: $\\chi$-integrated linecuts, $\\chi$-width normed', y=0.91, fontsize=14)
    fig.suptitle(f'{DA.film} $\\chi$-integrated linecuts, $\\chi$-width normed', y=0.91, fontsize=14)

    # fig.savefig(savePath.joinpath(f'sinchi-folded_{DA.film}_q{q_min}to{q_max}.png'), dpi=150, bbox_inches='tight')

    plt.show()
    plt.close('all')

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Set lims
oop_chi_min = 8
oop_chi_max = 30

ip_chi_min = 65
ip_chi_max = 87

q_min = None
q_max = 2.04

# Set savePath
savePath = outPath.joinpath('A1-3set_waxs/bcy651/linecut_plots_v1')

selected_attrs_dict = {}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)

for DA in tqdm(selected_DAs):
    # OOP
    OOP_linecut_DA = DA.sel(chi=slice(oop_chi_min, oop_chi_max), qr=slice(q_min,q_max)).integrate('chi')
    OOP_linecut_DA.plot.line(figsize=(6,4), label=f'Out-of-plane, {oop_chi_min} to {oop_chi_max}°' + r' $\chi$')

    # A plot.line xarray plot does not return an AxesImage object like imshow does
    # Can use plt.gca() and plt.gcf() to access the axes & figure parameters
    ax = plt.gca()
    fig = plt.gcf()

    # IP
    ax2 = ax.twinx()
    IP_linecut_DA = DA.sel(chi=slice(ip_chi_min, ip_chi_max), qr=slice(q_min,q_max)).integrate('chi')
    IP_linecut_DA.plot.line(color='C1', ax=ax2, label=f'In-plane, {ip_chi_min} to {ip_chi_max}°' + r' $\chi$')
    
    ax.set(title=f'{DA.film} 22° Linecuts, {float(DA.incident_angle[2:])}° Incidence',
           ylabel='Out-of-plane Intensity [arb. units]', xlabel='q$_r$ [Å$^{-1}$]')
    ax2.set(ylabel='In-plane Intensity [arb. units]')
    
    ax.grid(visible=True, which='major', axis='x')
    fig.set(tight_layout=True, dpi=130)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1))
    ax2.legend(loc='upper center', bbox_to_anchor=(0.5, 0.91)) 
    
    ax.figure.savefig(savePath.joinpath(f'sinchi-folded_{DA.film}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)

    # plt.show()
    plt.close('all')

### 1D line fitting*
Use lmfit to perform the linefits

In [ ]:
from scipy import optimize, signal
from lmfit import models

In [ ]:
def fit_plotter(q, I, out):
    """
    Plots a figure with 2 axes, the full fit on a log scale on the left and the components in normal scale on the right
    """
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.set(size_inches=(12,4))
    
    ax1.plot(q, I, label='data')
    ax1.plot(q, out.best_fit, label=f'full_fit, $R^2$={np.round(out.rsquared, 2)},' + r' $\chi^2_{red}$=' + f'{np.round(out.redchi, 1)}')
    ax1.set(xlabel='Q [Å$^{-1}$]', ylabel='Intensity [arb. units]')
    ax1.legend(loc='upper left')
    
    ax2.plot(q, I, label='data')
    ax2.plot(q, out.best_fit, label='full_fit')
    for key in out.eval_components():
        ax2.plot(q, out.eval_components()[key], label=f'{key}')
    ax2.set(xlabel='Q [Å$^{-1}$]')
    ax2.legend(loc='upper left')
    
    return (ax1, ax2), fig

#### Messy line fitting below

In [ ]:
avg_thicknesses = {
    'A1': 50,
    'A2': 40,
    'A3': 40
}

low_thicknesses = {
    'A1': 45,
    'A2': 30,
    'A3': 35
}

high_thicknesses = {
    'A1': 65,
    'A2': 70,
    'A3': 45
}


thicknesses = high_thicknesses

In [ ]:
### A1

# Set lims
oop_chi_min = 10
oop_chi_max = 30

ip_chi_min = 65
ip_chi_max = 82

q_min = 0.1
q_max = 2.04
# q_min = 0.2
# q_max = 0.8

# Set savePath
savePath = outPath.joinpath('A1-3set_waxs/bcy651/line_fits_v1')

selected_attrs_dict = {'film': ['A1']}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)


DA = selected_DAs[1].copy()
DA.data = DA.data / thicknesses['A1']  # thickness normed!

OOP_linecut_DA = DA.sel(chi=slice(oop_chi_min, oop_chi_max), qr=slice(0.3, 1.9)).integrate('chi')

point_x = 0.3
point_y = float(DA.sel(chi=slice(oop_chi_min, oop_chi_max), qr=slice(point_x-0.01, point_x+0.01)).mean('qr').integrate('chi'))
point_y = point_y - point_y*(0.08)

x = OOP_linecut_DA.qr.data
y = OOP_linecut_DA.data

# Define all models to include in fitting
bkg_mod = models.LinearModel(prefix='bkg_')
pars = bkg_mod.make_params()
pars = bkg_mod.make_params(intercept=point_y, slope=0)
pars['bkg_intercept'].set(vary=False)
pars['bkg_slope'].set(vary=False)

pk1_mod = models.PseudoVoigtModel(prefix='pk1_')
# pars = pk1_mod.make_params(center=0.55)
pars += pk1_mod.guess(y, x, center=0.55)
# pars['pk1_amplitude'].set(min=0.5)
pars['pk1_amplitude'].set(min=0)

# pk2_mod = models.LorentzianModel(prefix='pk2_')
pk2_mod = models.PseudoVoigtModel(prefix='pk2_')

pars += pk2_mod.guess(y, x, center=1.7)
# pars['pk2_center'].set(max = 0.62, min = 0.58)
# pars['pk2_sigma'].set(max = 0.03)
pars['pk2_amplitude'].set(min=0)

pk3_mod = models.PseudoVoigtModel(prefix='pk3_')
pars += pk3_mod.guess(y, x, center=0.8)
pars['pk3_center'].set(max=0.9, min = 0.7)
pars['pk3_amplitude'].set(min=0)


# Combine into full model
mod = bkg_mod + pk1_mod + pk2_mod + pk3_mod

# Run fit and store all info in a ModelResult object
out = mod.fit(y, pars, x=x)
FWHM = np.round(float(out.params['pk2_fwhm']), 2)
center = np.round(float(out.params['pk2_center']), 2)
Lc = np.round((2*np.pi*0.9)/FWHM, 2)

# Plot
axs, fig = fit_plotter(x, y, out)
fig.set(size_inches=(10,4), tight_layout=True, dpi=120)

axs[0].set(ylabel=f'Integrated intensity [arb. units]', title=f'Scherrer coherence length = {Lc} Å')
axs[1].set(title=f'$\pi$-$\pi$ peak: FWHM = {FWHM}' + ' Å$^{-1}$, ' + f'peak center = {center}' + ' Å$^{-1}$')
fig.suptitle(f'{DA.film} film, sin($\chi$)-corrected linecut fit, {oop_chi_min}° to {oop_chi_max}° $\chi$, {DA.incident_angle[2:-1]}° incident angle')    

# fig.savefig(savePath.joinpath(f'{DA.film}_{intensity_type}_{DA.incident_angle}_{oop_chi_min}to{oop_chi_max}chi.png'), dpi=120)
plt.show()
plt.close('all')


print(f'FWHM = {FWHM}, Lc = {Lc}\n')
display(out.best_values)

In [ ]:
### A2

# Set lims
oop_chi_min = 10
oop_chi_max = 30

ip_chi_min = 65
ip_chi_max = 82

q_min = 0.1
q_max = 2.04
# q_min = 0.2
# q_max = 0.8

# Set savePath
savePath = outPath.joinpath('A1-3set_waxs/bcy651/line_fits_v1')

selected_attrs_dict = {'film': ['A2']}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)


DA = selected_DAs[1].copy()
DA.data = DA.data / thicknesses['A2']

OOP_linecut_DA = DA.sel(chi=slice(oop_chi_min, oop_chi_max), qr=slice(0.3, 1.9)).integrate('chi')

point_x = 0.3
point_y = float(DA.sel(chi=slice(oop_chi_min, oop_chi_max), qr=slice(point_x-0.01, point_x+0.01)).mean('qr').integrate('chi'))
point_y = point_y - point_y*(0.08)

x = OOP_linecut_DA.qr.data
y = OOP_linecut_DA.data

# Define all models to include in fitting
bkg_mod = models.LinearModel(prefix='bkg_')
pars = bkg_mod.make_params()
pars = bkg_mod.make_params(intercept=point_y, slope=0)
pars['bkg_intercept'].set(vary=False)
pars['bkg_slope'].set(vary=False)

pk1_mod = models.PseudoVoigtModel(prefix='pk1_')
# pars = pk1_mod.make_params(center=0.55)
pars += pk1_mod.guess(y, x, center=0.55)
# pars['pk1_amplitude'].set(min=0.5)
pars['pk1_amplitude'].set(min=0)

# pk2_mod = models.LorentzianModel(prefix='pk2_')
pk2_mod = models.PseudoVoigtModel(prefix='pk2_')

pars += pk2_mod.guess(y, x, center=1.7)
# pars['pk2_center'].set(max = 0.62, min = 0.58)
# pars['pk2_sigma'].set(max = 0.03)
pars['pk2_amplitude'].set(min=0)

# pk3_mod = models.PseudoVoigtModel(prefix='pk3_')
# pars += pk3_mod.guess(y, x, center=0.8)
# pars['pk3_center'].set(max=0.9, min = 0.7)
# pars['pk3_amplitude'].set(min=0)


# Combine into full model
# mod = bkg_mod + pk1_mod + pk2_mod + pk3_mod
mod = bkg_mod + pk1_mod + pk2_mod  # + pk3_mod

# Run fit and store all info in a ModelResult object
out = mod.fit(y, pars, x=x)
FWHM = np.round(float(out.params['pk2_fwhm']), 2)
center = np.round(float(out.params['pk2_center']), 2)
Lc = np.round((2*np.pi*0.9)/FWHM, 2)

# Plot
axs, fig = fit_plotter(x, y, out)
fig.set(size_inches=(10,4), tight_layout=True, dpi=120)

axs[0].set(ylabel=f'Integrated intensity [arb. units]', title=f'Scherrer coherence length = {Lc} Å')
axs[1].set(title=f'$\pi$-$\pi$ peak: FWHM = {FWHM}' + ' Å$^{-1}$, ' + f'peak center = {center}' + ' Å$^{-1}$')
fig.suptitle(f'{DA.film} film, sin($\chi$)-corrected linecut fit, {oop_chi_min}° to {oop_chi_max}° $\chi$, {DA.incident_angle[2:-1]}° incident angle')    

# fig.savefig(savePath.joinpath(f'{DA.film}_{intensity_type}_{DA.incident_angle}_{oop_chi_min}to{oop_chi_max}chi.png'), dpi=120)
plt.show()
plt.close('all')


print(f'FWHM = {FWHM}, Lc = {Lc}\n')
display(out.best_values)

In [ ]:
### A3

# Set lims
oop_chi_min = 10
oop_chi_max = 30

ip_chi_min = 65
ip_chi_max = 82

q_min = 0.1
q_max = 2.04
# q_min = 0.2
# q_max = 0.8

# Set savePath
savePath = outPath.joinpath('A1-3set_waxs/bcy651/line_fits_v1')

selected_attrs_dict = {'film': ['A3']}
selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)


DA = selected_DAs[1].copy()
DA.data = DA.data / thicknesses['A3']

OOP_linecut_DA = DA.sel(chi=slice(oop_chi_min, oop_chi_max), qr=slice(0.3, 1.9)).integrate('chi')

point_x = 0.3
point_y = float(DA.sel(chi=slice(oop_chi_min, oop_chi_max), qr=slice(point_x-0.01, point_x+0.01)).mean('qr').integrate('chi'))
point_y = point_y - point_y*(0.08)

x = OOP_linecut_DA.qr.data
y = OOP_linecut_DA.data

# Define all models to include in fitting
bkg_mod = models.LinearModel(prefix='bkg_')
pars = bkg_mod.make_params()
pars = bkg_mod.make_params(intercept=point_y, slope=0)
pars['bkg_intercept'].set(vary=False)
pars['bkg_slope'].set(vary=False)

pk1_mod = models.PseudoVoigtModel(prefix='pk1_')
# pars = pk1_mod.make_params(center=0.55)
pars += pk1_mod.guess(y, x, center=0.55)
# pars['pk1_amplitude'].set(min=0.5)
pars['pk1_amplitude'].set(min=0)

# pk2_mod = models.LorentzianModel(prefix='pk2_')
pk2_mod = models.PseudoVoigtModel(prefix='pk2_')

pars += pk2_mod.guess(y, x, center=1.7)
# pars['pk2_center'].set(max = 0.62, min = 0.58)
# pars['pk2_sigma'].set(max = 0.03)
pars['pk2_amplitude'].set(min=0)

# pk3_mod = models.PseudoVoigtModel(prefix='pk3_')
# pars += pk3_mod.guess(y, x, center=0.8)
# pars['pk3_center'].set(max=0.9, min = 0.7)
# pars['pk3_amplitude'].set(min=0)


# Combine into full model
# mod = bkg_mod + pk1_mod + pk2_mod + pk3_mod
mod = bkg_mod + pk1_mod + pk2_mod #+ pk3_mod

# Run fit and store all info in a ModelResult object
out = mod.fit(y, pars, x=x)
FWHM = np.round(float(out.params['pk2_fwhm']), 2)
center = np.round(float(out.params['pk2_center']), 2)
Lc = np.round((2*np.pi*0.9)/FWHM, 2)

# Plot
axs, fig = fit_plotter(x, y, out)
fig.set(size_inches=(10,4), tight_layout=True, dpi=120)

axs[0].set(ylabel=f'Integrated intensity [arb. units]', title=f'Scherrer coherence length = {Lc} Å')
axs[1].set(title=f'$\pi$-$\pi$ peak: FWHM = {FWHM}' + ' Å$^{-1}$, ' + f'peak center = {center}' + ' Å$^{-1}$')
fig.suptitle(f'{DA.film} film, sin($\chi$)-corrected linecut fit, {oop_chi_min}° to {oop_chi_max}° $\chi$, {DA.incident_angle[2:-1]}° incident angle')    

# fig.savefig(savePath.joinpath(f'{DA.film}_{intensity_type}_{DA.incident_angle}_{oop_chi_min}to{oop_chi_max}chi.png'), dpi=120)
plt.show()
plt.close('all')


print(f'FWHM = {FWHM}, Lc = {Lc}\n')
display(out.best_values)

### Pole Figures

#### Pi-pi 

In [ ]:
### Set limits
q_max = 2
q_min = 1.4
chi_min = 10
chi_max = 45  # to ~45 least until I mask out the silicon(?) background scattering that interferes

### Set savePath
savePath = outPath.joinpath('A1-3set_waxs/pipi_pole_figures_v1')

### Select dataarrays to plot

selected_attrs_dict = {'film':['A1', 'A2', 'A3']}
# selected_attrs_dict = {'film':['D18p1', 'D18A1p1', 'D18A2p1', 'D18A3'],
#                        'incident_angle':['th0.140']}
# selected_attrs_dict = {}

selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)
 
peak_areas_dict = {}    
for DA in tqdm(selected_DAs):
    chis = DA.chi.sel(chi=slice(chi_min,chi_max)).data
    
    peak_areas = np.array([])
    peak_centers = np.array([])
    
    for chi in chis:
        sliced_DA = DA.sel(qr=slice(q_min, q_max), chi=float(chi))

        points_x = [q_min, q_max]
        points_y = [float(sliced_DA.sel(qr=slice(points_x[0]-0.1, points_x[0]+0.1)).mean('qr')), 
                    float(sliced_DA.sel(qr=slice(points_x[1]-0.1, points_x[1]+0.1)).mean('qr'))]
        m = (points_y[1]-points_y[0])/(points_x[1]-points_x[0])
        # print(m)
        b = points_y[1] - (m*points_x[1])
        y_fit = np.polyval([m, b], sliced_DA.qr)
        
        sub_sliced_DA = sliced_DA-y_fit

        peak_area = sub_sliced_DA.sel(qr=slice(1.5,1.8)).integrate('qr')
        if peak_area<0 or m>50 or m<-50:
            peak_area = np.nan
        # peak_area = np.nan if peak_area<0 else peak_area  # remove negatives as they are nonphysical
        
        peak_areas = np.append(peak_areas, float(peak_area))
                        
#         # Plot linecuts for each chi
#         fig, ax = plt.subplots()
#         sliced_DA.plot(ax=ax)
#         ax.plot(points_x, points_y)
#         plt.show()

#         plt.close('all')

    normed_peak_areas = peak_areas / np.nansum(peak_areas)
    data = np.vstack((chis, normed_peak_areas)).T
    peak_areas_dict[DA.attrs['film']+'-'+DA.attrs['incident_angle']] = data
    
    # Save pipi pole figure text files
    txtPath = outPath.joinpath('A1-3set_waxs/bcy651/text_files/pipi_pole_figures')
    np.savetxt(txtPath.joinpath(f'{DA.film}_{DA.incident_angle}.txt'), data)

    # Individual measurement pole figure plot
    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    ax.plot(chis, normed_peak_areas)
    fig.suptitle(f'{DA.film} Pole Figure; $\\alpha_i = $ {DA.incident_angle[2:-1]}°', x=0.54)
    ax.set(title=f'{q_min} to {q_max} 1/Å Q Bounds; {chi_min}° to {chi_max}° $\chi$')
    ax.set(xlabel='$\chi$ [°]', ylabel='Integrated Pi-Pi Peak Area [arb. units]')
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    
    # fig.savefig(savePath.joinpath(f'{DA.film}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    
    plt.show()
    
    plt.close('all')

In [43]:
data

array([[1.05000180e+01, 6.50889850e-02],
       [1.15000181e+01, 6.81618339e-02],
       [1.25000183e+01, 6.90906274e-02],
       [1.35000185e+01, 6.91907768e-02],
       [1.45000187e+01, 6.53910652e-02],
       [1.55000189e+01, 6.55540621e-02],
       [1.65000190e+01, 6.01170068e-02],
       [1.75000192e+01, 5.55585920e-02],
       [1.85000194e+01, 5.64295986e-02],
       [1.95000196e+01, 5.14578860e-02],
       [2.05000198e+01, 4.78240765e-02],
       [2.15000199e+01, 4.17997652e-02],
       [2.25000201e+01, 4.11653239e-02],
       [2.35000203e+01, 3.45436225e-02],
       [2.45000205e+01, 2.94976039e-02],
       [2.55000207e+01, 2.78441210e-02],
       [2.65000208e+01, 2.37575395e-02],
       [2.75000210e+01, 2.26214315e-02],
       [2.85000212e+01, 1.89545370e-02],
       [2.95000214e+01, 1.88046915e-02],
       [3.05000215e+01, 1.38552806e-02],
       [3.15000217e+01, 1.29219713e-02],
       [3.25000219e+01, 1.19824124e-02],
       [3.35000221e+01, 9.26368454e-03],
       [3.450002

In [ ]:
savePath.exists()

In [ ]:
### Set savePath
savePath = outPath.joinpath('A1-3set_waxs/bcy651/pipi_pole_figures_v1')

for th in tqdm(['th0.100', 'th0.120', 'th0.140']):
    selected_dict_items = [(name, data) for (name, data) in peak_areas_dict.items() if 
                           # name.split('-')[0] in ['D18p1', 'D18A1p1', 'D18A2p1', 'D18A3'] and 
                           # name.split('-')[0] in ['D18p2', 'D18A1p2', 'D18A2p2', 'D18A3'] and 
                           name.split('-')[0] in ['A1', 'A2', 'A3'] and 
                           # 'D18' in name and
                           th in name]    

    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    colors = plt.cm.Dark2(np.linspace(0,1,len(selected_dict_items)))

    for i, (measurement, pole_fig_arr_data) in enumerate(selected_dict_items):
        chis = pole_fig_arr_data[:,0]
        normed_peak_areas = pole_fig_arr_data[:,1]
        ax.plot(chis, normed_peak_areas, color=colors[i], label=measurement)
        ax.set(title=f'Normalized Pi-Pi Pole Figure: {chi_min}° to {chi_max}° $\chi$')
        ax.set(xlabel='$\chi$ [°]', ylabel='Normalized Peak Area')
    
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    ax.legend(loc='upper right')
    
    # fig.savefig(savePath.joinpath(f'multiple-D18all_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)
    # fig.savefig(savePath.joinpath(f'multiple-D18p1s_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)
    # fig.savefig(savePath.joinpath(f'multiple-D18p2s_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)
    fig.savefig(savePath.joinpath(f'multiple-A123_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)

    
    plt.show()
    plt.close('all')

#### Lamella

In [ ]:
### Set limits
q_min = 0.24
q_max = 0.5
chi_min = 2
chi_max = 87  # to ~45 least until I mask out the silicon(?) background scattering that interferes

### Set savePath
savePath = outPath.joinpath('A1-3set_waxs/lamella_pole_figures_v1')

### Select dataarrays to plot

# selected_attrs_dict = {'film':['A2', 'A1', 'A3'],
#                        'incident_angle':['th0.120']}
# selected_attrs_dict = {'film':['D18p1', 'D18A1p1', 'D18A2p1', 'D18A3'],
#                        'incident_angle':['th0.140']}
# selected_attrs_dict = {'film':['D18p1'],
#                        'incident_angle':['th0.120']}
selected_attrs_dict = {}

selected_DAs = select_attrs(folded_corr_DAs, selected_attrs_dict)
 
peak_areas_dict = {}    
for DA in tqdm(selected_DAs):
    chis = DA.chi.sel(chi=slice(chi_min,chi_max)).data
    
    peak_areas = np.array([])
    peak_centers = np.array([])
    
    for chi in chis:
        sliced_DA = DA.sel(qr=slice(q_min, q_max), chi=float(chi))

        points_x = [q_min, q_max]
        # points_y = [float(sliced_DA.sel(qr=slice(points_x[0]-1e-3, points_x[0]+1e-3)).mean('qr')), 
        #             float(sliced_DA.sel(qr=slice(points_x[1]-1e-3, points_x[1]+1e-3)).mean('qr'))]
        points_y = [float(sliced_DA.sel(qr=q_min, method='nearest')), 
                    float(sliced_DA.sel(qr=q_max, method='nearest'))]
        m = (points_y[1]-points_y[0])/(points_x[1]-points_x[0])
        b = points_y[1] - (m*points_x[1])
        y_fit = np.polyval([m, b], sliced_DA.qr)
        
        sub_sliced_DA = sliced_DA-y_fit

        peak_area = sub_sliced_DA.sel(qr=slice(None,0.45)).integrate('qr')
        peak_area = np.nan if peak_area<0 else peak_area  # remove negatives as they are nonphysical
        
        peak_areas = np.append(peak_areas, float(peak_area))
                        
#         # Plot linecuts for each chi
#         fig, ax = plt.subplots()
#         sliced_DA.plot(ax=ax)
#         ax.plot(points_x, points_y)
#         plt.show()

#         plt.close('all')

    normed_peak_areas = peak_areas / np.nansum(peak_areas)
    data = np.vstack((chis, normed_peak_areas)).T
    peak_areas_dict[DA.attrs['film']+'-'+DA.attrs['incident_angle']] = data

    # Individual measurement pole figure plot
    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    ax.plot(chis, normed_peak_areas)
    fig.suptitle(f'{DA.film} Pole Figure; $\\alpha_i = $ {DA.incident_angle[2:-1]}°', x=0.54)
    ax.set(title=f'{q_min} to {q_max} 1/Å Q Bounds; {chi_min}° to {chi_max}° $\chi$')
    ax.set(xlabel='$\chi$ [°]', ylabel='Integrated Pi-Pi Peak Area [arb. units]')
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    
    # fig.savefig(savePath.joinpath(f'{DA.film}_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{DA.incident_angle}.png'), dpi=150)
    
    plt.show()
    
    plt.close('all')

In [ ]:
### Set savePath
savePath = outPath.joinpath('A1-3set_waxs/lamella_pole_figures_v1')

for th in tqdm(['th0.100', 'th0.120', 'th0.140']):
    selected_dict_items = [(name, data) for (name, data) in peak_areas_dict.items() if 
                           'D18' in name and
                           # name.split('-')[0] in ['D18p1', 'D18A1p1', 'D18A2p1', 'D18A3'] and 
                           # name.split('-')[0] in ['D18p2', 'D18A1p2', 'D18A2p2', 'D18A3'] and 
                           # name.split('-')[0] in ['A1', 'A2', 'A3'] and 
                           th in name]    

    fig, ax = plt.subplots(figsize=(5,3.3))
    fig.set(dpi=120)
    
    colors = plt.cm.Dark2(np.linspace(0,1,len(selected_dict_items)))

    for i, (measurement, pole_fig_arr_data) in enumerate(selected_dict_items):
        chis = pole_fig_arr_data[:,0]
        normed_peak_areas = pole_fig_arr_data[:,1]
        ax.plot(chis, normed_peak_areas, color=colors[i], label=measurement)
        fig.suptitle('Pole Figure', x=0.55)
        ax.set(title=f'{q_min} to {q_max} 1/Å Q Fit Region ; {chi_min}° to {chi_max}° $\chi$')
        ax.set(xlabel='$\chi$ [°]', ylabel='Normalized Peak Area')
    
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.2)
    ax.legend(loc='upper left')
    
    # fig.savefig(savePath.joinpath(f'multiple-D18all_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)
    # fig.savefig(savePath.joinpath(f'multiple-D18p1s_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)
    # fig.savefig(savePath.joinpath(f'multiple-D18p2s_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)
    # fig.savefig(savePath.joinpath(f'multiple-A123_chi{chi_min}to{chi_max}_q{q_min}to{q_max}_{th}.png'), dpi=150)

    
    plt.show()
    plt.close('all')

# Potentially useful cells:

In [ ]:
# Get unique attribute names quickly
set([da.attrs['film'] for da in folded_corr_DAs])

In [ ]:
# ### A way to save data as csv files  
# for DA in DS.data_vars.values():
#     # qr columns, chi rows
#     DA.to_pandas().to_csv(outPath.joinpath('PM6-Y6_waxs', f'polar-2D_{DA.polymer}-{DA.weight_percent}_{DA.incident_angle}_{DA.scan_id}.csv'))

In [ ]:
# ### Old pole figure code
# q_max = 2.45
# q_min = 1.1
# chi_min = -30
# chi_max = 40

# intensity_type = 'corr'

# if intensity_type == 'raw':
#     selected_DAs = selected_raw_DAs
# elif intensity_type == 'corr':
#     selected_DAs = selected_corr_DAs
    
# for DA in tqdm(selected_DAs):
#     chis = DA.chi.sel(chi=slice(chi_min,chi_max)).data
    
#     peak_areas = np.array([])
#     peak_centers = np.array([])
    
#     for chi in tqdm(chis):
#         sliced_DA = DA.sel(qr=slice(q_min, q_max), chi=float(chi))

#         points_x = [1.15, 2.3]
#         points_y = [float(sliced_DA.sel(qr=slice(points_x[0]-0.05, points_x[0]+0.05)).mean('qr')), 
#                     float(sliced_DA.sel(qr=slice(points_x[1]-0.2, points_x[1]+0.2)).mean('qr'))]
#         m = (points_y[1]-points_y[0])/(points_x[1]-points_x[0])
#         b = points_y[1] - (m*points_x[1])
#         y_fit = np.polyval([m, b], sliced_DA.qr)
        
#         sub_sliced_DA = sliced_DA-y_fit

#         peak_area = sub_sliced_DA.sel(qr=slice(1.5,1.8)).integrate('qr')
#         peak_areas = np.append(peak_areas, float(peak_area))
        
#         # Run lmfit per chi around pi-pi if desired
#         # Build model 
#         x = sub_sliced_DA.qr.data
#         y = sub_sliced_DA.data
#         pk1_mod = models.PseudoVoigtModel(prefix='pk1_')
#         pars = pk1_mod.make_params(center=1.7)
#         pars += pk1_mod.guess(y, x, center=1.7)
#         # pars['pk1_amplitude'].set(min=0.5)

#         # Run fit
#         try: 
#             out = pk1_mod.fit(y, pars, x=x)
#             peak_centers = np.append(peak_centers, np.round(float(out.params['pk1_center']), 2))
#         except ValueError:
#             peak_centers = np.append(peak_centers, np.nan)
            
        
# #         # Plot linecuts for each chi
# #         fig, ax = plt.subplots()
# #         sliced_DA.plot(ax=ax)
# #         ax.plot(points_x, points_y)
# #         plt.show()
        
# # #         axs, fig = fit_plotter(x, y, out)        
# # #         sub_sliced_DA.plot(ax=ax)

# # #         ax.set(xlim=(q_min,q_max))
# # #         plt.show()

# #         plt.close('all')

#     # fig, axs = plt.subplots(1,2, figsize=(12,4))
#     # axs[0].plot(chis, peak_areas)
#     # axs[1].plot(chis, peak_centers)
    
#     # Pole figure plot
#     fig, ax = plt.subplots(figsize=(7.5,4))
#     ax.plot(chis, peak_areas)
#     if intensity_type == 'corr':
#         ax.set(title=f'{DA.film} Film Pole Figure, {DA.incident_angle[2:-1]}° Incidence, Sin($\chi$)-Corrected, {chi_min}° to {chi_max}° $\chi$')
#     elif intensity_type =='raw':
#         ax.set(title=f'{DA.film} Film Pole Figure, {DA.incident_angle[2:-1]}° Incidence, Raw Intensity: {chi_min}° to {chi_max}° $\chi$')
#     ax.set(xlabel='$\chi$ [°]', ylabel='Integrated Pi-Pi Peak Area [arb. units]')
#     # fig.savefig(outPath.joinpath('A1-3set_waxs/pole_figures', f'{DA.film}_{intensity_type}_{DA.incident_angle}_{chi_min}to{chi_max}chi.png'), dpi=120)
#     plt.show()
    
#     # Peak center plot
#     fig, ax = plt.subplots(figsize=(7.5,4))
#     ax.plot(chis, peak_centers)
#     if intensity_type == 'corr':
#         ax.set(title=f'{DA.film} Film Pi-Pi Peak Center, {DA.incident_angle[2:-1]}° Incidence, Sin($\chi$)-Corrected, {chi_min}° to {chi_max}° $\chi$')
#     elif intensity_type =='raw':
#         ax.set(title=f'{DA.film} Film Pi-Pi Peak Center, {DA.incident_angle[2:-1]}° Incidence, Raw Intensity: {chi_min}° to {chi_max}° $\chi$')
#     ax.set(xlabel='$\chi$ [°]', ylabel='Pi-Pi Peak Center [Å$^{-1}$]')
#     # fig.savefig(outPath.joinpath('A1-3set_waxs/pole_figures', f'{DA.film}_{intensity_type}_peak-center_{DA.incident_angle}_{chi_min}to{chi_max}chi.png'), dpi=120)
#     plt.show()    
    
    
#     plt.close('all')